In [1]:
pip install sentence-transformers numpy pandas scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import numpy as np
import pandas as pd
import time
import torch
from sentence_transformers import SentenceTransformer, util
from scipy.stats import rankdata
import os

models = [
    "all-MiniLM-L6-v2",
    "paraphrase-MiniLM-L3-v2",
    "distilbert-base-nli-mean-tokens",
    "all-roberta-large-v1",
    "sentence-t5-base",
    "bert-base-nli-mean-tokens",
    "stsb-roberta-large"
]

sentences1 = [
    "The cat sits on the mat.",
    "Artificial Intelligence is transforming the world."
]
sentences2 = [
    "A feline is resting on a rug.",
    "AI is changing everything around us."
]

data = []

for model_name in models:
    model = SentenceTransformer(model_name)

    start_time = time.time()
    embeddings1 = model.encode(sentences1, convert_to_tensor=True)
    embeddings2 = model.encode(sentences2, convert_to_tensor=True)
    inference_time = time.time() - start_time

    cosine_similarities = util.pytorch_cos_sim(embeddings1, embeddings2).mean().item()

    euclidean_distance = torch.norm(embeddings1 - embeddings2, p=2).mean().item()
    manhattan_distance = torch.norm(embeddings1 - embeddings2, p=1).mean().item()

    processing_speed = len(sentences1) / inference_time

    memory_size = sum(p.numel() for p in model.parameters()) * 4 / (1024 * 1024)

    model_save_path = model_name.replace("/", "_")
    model.save(model_save_path)
    disk_size = sum(os.path.getsize(os.path.join(model_save_path, f)) for f in os.listdir(model_save_path)) / (1024 * 1024)
    os.system(f"rm -rf {model_save_path}")

    data.append([model_name, cosine_similarities, inference_time, memory_size, euclidean_distance, manhattan_distance, processing_speed, disk_size])

df = pd.DataFrame(data, columns=["Model", "Cosine Similarity", "Inference Time (s)", "Memory Usage (MB)",
                                 "Euclidean Distance", "Manhattan Distance", "Processing Speed (sent/sec)", "Disk Size (MB)"])

X = df.iloc[:, 1:].values
X_norm = X / np.sqrt((X ** 2).sum(axis=0))

weights = np.array([0.3, 0.2, 0.1, -0.15, -0.15, 0.2, -0.1])
criteria = np.array([1, -1, -1, -1, -1, 1, -1])

ideal_best = np.max(X_norm * criteria, axis=0)
ideal_worst = np.min(X_norm * criteria, axis=0)

distance_best = np.sqrt(((X_norm - ideal_best) ** 2).sum(axis=1))
distance_worst = np.sqrt(((X_norm - ideal_worst) ** 2).sum(axis=1))

topsis_score = distance_worst / (distance_best + distance_worst)

df["TOPSIS Score"] = topsis_score
df["Rank"] = rankdata(-topsis_score, method='dense')

print(df.sort_values(by="Rank"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

                             Model  Cosine Similarity  Inference Time (s)  \
1          paraphrase-MiniLM-L3-v2           0.247923            0.036366   
2  distilbert-base-nli-mean-tokens           0.512723            0.218735   
4                 sentence-t5-base           0.763034            0.397373   
5        bert-base-nli-mean-tokens           0.413606            0.345992   
0                 all-MiniLM-L6-v2           0.307084            0.267321   
6               stsb-roberta-large           0.374480            1.049815   
3             all-roberta-large-v1           0.413923            1.081144   

   Memory Usage (MB)  Euclidean Distance  Manhattan Distance  \
1          66.336914            7.323734          160.563522   
2         253.154297           16.259890          501.539917   
4         420.449707            0.628915           19.414396   
5         417.641602           17.955481          553.319458   
0          86.644043            1.176970           25.800083   